In [1]:
!pip install 'stable-baselines3[extra]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.5 MB/s eta 0:00:00
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446661 sha256=e82a7a800891ec809d609d2e27283db0e15fb45636e8a71f71bb9269ff407c88
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [2]:
import gym
from stable_baselines3 import DQN , PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.callbacks import EvalCallback

#from stable_baselines3.common.noise import LinearSchedule

In [3]:
import os
from gym import Env, spaces
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import rc
import matplotlib.animation as animation
from IPython.display import HTML, Image
from scipy import integrate
import time

In [4]:
class simulator(Env):
    def __init__(self, n_reactors=9, action_size=2, state_size=1, volume=1):
        self.volume = volume
        self.reactor_seq = []
        self.X = [0]
        self.state = None
        #self.action_size = action_size
        #self.state_size = n_reactors + 1 #1 for "done" for discrete state
        #self.state_size = state_size
        self.n_reactors = n_reactors
        self.action_space = spaces.Discrete(action_size)
        self.observation_space = spaces.Box(low=0, high=1, shape = (1,))
        self.n_steps = 0



    def choose_reactor(self, action):
        self.reactor_seq.append(action)

    def step(self, action):
        self.n_steps += 1
        self.choose_reactor(action)
        self.X.append(self.equation_solver(action, self.X[-1]))
        self.state = np.array([self.X[-1]])  # seems like most envs give state as a np array
        reward = self.X[-1] - self.X[-2]  #increase in conversion
        if self.n_steps == self.n_reactors or self.X[-1] >= 1:
            done = True
        else:
            done = False
        return self.state, reward, done, {}

    def show_seq(self):
        return self.reactor_seq, self.X


    def reset(self):
        self.n_steps = 0
        self.reactor_seq = []
        self.X = [0]
        self.state = np.array([0])
        return self.state

    def render(self, mode='human'):
        print(f'choice({len(self.X)-1}) - conversions: {self.X}, reactors: {self.reactor_seq}')

    def equation_solver(self, r_type, X_prev, a=-10, b=10, c=2):
        Vol = self.volume
        X_new = X_prev
        # see page 13 of report
        if r_type == 0: # CSTR
            A = a
            B = b - a*X_prev
            C = c - b*X_prev
            D = -c*X_prev - Vol
            coeffs = [A, B, C, D]
        else:
            A = a/3
            B = b/2
            C = c
            D = -(a/3 * X_prev**3 + b/2*X_prev**2 + c* X_prev) - Vol
            coeffs = [A, B, C, D]

        roots = np.roots(coeffs)

        if True in np.isreal(roots):
            roots = roots[np.isreal(roots)]
            pos_roots = np.array([root for root in roots if root>X_prev])
            if pos_roots.size > 0:
                diffs = pos_roots - X_prev  #closest conversion solution above X prev
                X_new = pos_roots[diffs == (min(diffs))][0]
            else:
                X_new = 1
        if X_new > 1:
            X_new = 1

        return X_new

In [5]:
env = simulator(volume = 0.8, n_reactors=4)
print("Action space:", env.action_space)


Action space: Discrete(2)


In [6]:
net_arch = [16,16,16]

# Create a Linear Schedule for epsilon decay
#exploration_schedule = LinearSchedule(schedule_timesteps=int(0.1 * steps), initial_p=1.0, final_p=0.05)

# Create and train the DQN agent
model = DQN("MlpPolicy", env, gamma=0.95, learning_rate=1e-3, buffer_size=50000,exploration_initial_eps = 1, exploration_final_eps=0.05,
          exploration_fraction=1, train_freq= 1 ,batch_size = 1, gradient_steps=1, policy_kwargs=dict(net_arch=[16,16,16]),learning_starts = 9,
          tensorboard_log="./dqn_simulator_tensorboard/")

# Train the DQN agent
model.learn(total_timesteps=100000)

# Test the trained agent
mean_reward, reactor_seq = evaluate_policy(model, env, n_eval_episodes=10)

print(f"Mean reward: {mean_reward}")
env.close()

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Mean reward: 0.9999999850988388


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [8]:
episode = 1
for episode in range(1,episode+1):
    obs = env.reset()
    done = False
    Conversion = 0

    while not done :
        action,_ = model.predict(obs)
        obs,reward,done,info = env.step(action)
        Conversion+=reward
        print("Action : {}, Conversion = {}".format(action,reward))
    print("Episode : {} Conversion : {}".format(episode,Conversion))
    #print("Reactor Sequencing = {}".format(env.reactor_seq))


Action : 1, Conversion = 0.2601494936395941
Action : 1, Conversion = 0.18789476616039946
Action : 0, Conversion = 0.1850643356860045
Action : 0, Conversion = 0.36689140451400193
Episode : 1 Conversion : 1.0


In [9]:
episode = 1
for episode in range(1,episode+1):
    obs = env.reset()
    done = False
    Conversion = 0

    while not done :
        action,_ = model.predict(obs)
        obs,reward,done,info = env.step(action)
        Conversion+=reward
        print("Action : {}, Conversion = {}".format(action,reward))
    print("Episode : {} Conversion : {}".format(episode,Conversion))
    #print("Reactor Sequencing = {}".format(env.reactor_seq))


Action : 1, Conversion = 0.2601494936395941
Action : 1, Conversion = 0.18789476616039946
Action : 0, Conversion = 0.1850643356860045
Action : 0, Conversion = 0.36689140451400193
Episode : 1 Conversion : 1.0


In [ ]:
env.close()